In [1]:
# Import usual Python and data handling stuff
import numpy as np
import pandas as pd

In [2]:
# Not yet used - No graph :-(
#import matplotlib.pyplot as plt
#%matplotlib inline

# Analysis of the participations in EU Framework Programmes

In the analysis, we cover the participation in the EU Framework Programmes for Research and Innovation between 2007 and 2017. This period is covered by FP7 (2007-2013) and Horizon 2020 (since 2014).

## FP7 datasets: 2007-2013

### Set up general coding information from Cordis

Cordis has several dictionaries for coded information. The following datasets are created on this basis and will be used to "decode" the information provided in the projects and organisations datasets for FP7.

In [3]:
# Symbols and names of countries in several langues
country_file = "FP7/cordisref-countries.xls"
countries = pd.read_excel(
    country_file,
    sheet_name="cordisref-countries",
    header=0,
)

In [4]:
# Symbols and names of FP7 sub-programmes in several langues
fp7_programmes_file = "FP7/cordisref-FP7programmes.xls"
fp7_programmes = pd.read_excel(
    fp7_programmes_file,
    sheet_name="Hoja1",
    header=0,
)

In [5]:
# Full names of funding schemes
fp7_schemes_file = "FP7/cordisref-projectFundingSchemeCategory.xls"
fp7_schemes = pd.read_excel(
    fp7_schemes_file,
    sheet_name="cordisref-projectFundingSchemeC",
    header=0,
)

### FP7 projects and organisations
The data for projects and participations in FP7 are available on the Open Data Portal of the European Union:

https://data.europa.eu/euodp/en/data/dataset/cordisfp7projects

Two datasets will be created:
* `fp7_proj`: with the descriptors of the FP7 funded projects
* `fp7_part`: with the descriptions of the participating organisations

These two datasets will be merged into one sigle `fp7` dataset where the project informations will be repeated to each participations.

In [6]:
%%time

fp7_proj_file = "FP7/cordis-fp7projects.xlsx"
fp7_part_file = "FP7/cordis-fp7organizations.xlsx"

fp7_proj = pd.read_excel(
    fp7_proj_file, sheet_name="cordis-fp7projects",
    header=0,
)
fp7_part = pd.read_excel(
    fp7_part_file,
    sheet_name="cordis-h2020organizations", # Which is a mistake in the Cordis dataset!
    header=0,
)

CPU times: user 20.8 s, sys: 112 ms, total: 20.9 s
Wall time: 20.9 s


In [7]:
# We  rename some columns which are in both datasets:
# In projects:
fp7_proj = fp7_proj.rename(
    index=str,
    columns={
        "id": "projectID",
        "rcn": "projectRCN",
        "acronym": "projectAcronym"
    }
)
# In participations:
fp7_part = fp7_part.rename(
    index=str,
    columns={
        "id": "organizationID",
        "projectRcn": "projectRCN",
        "name": "organizationName",
    }
)

Some project acronyms are not consistent in their writing (lower case), so we force upper case writing:

In [8]:
fp7_proj["projectAcronym"] = fp7_proj["projectAcronym"].str.upper()
fp7_part["projectAcronym"] = fp7_part["projectAcronym"].str.upper()

So, let's merge these two datasets and create the `fp7` dataset.

In [9]:
fp7 = fp7_proj.merge(
    fp7_part,
    on=["projectRCN","projectID","projectAcronym"]
)

In [10]:
fp7_proj.shape, fp7_part.shape, fp7.shape

((25778, 21), (146021, 23), (146021, 41))

In [11]:
# We won't use these dataframes anymore
fp7_part = 0
fp7_proj = 0

## Horizon 2020 datasets: 2014-2020

### Set up general coding information from Cordis

Cordis has several dictionaries for coded information. The following datasets are created on this basis and will be used to "decode" the information provided in the projects and organisations datasets for FP7 and Horizon 2020.

In [12]:
# Symbols and names of Horizon 2020 sub-programmes in several languages
h2020_programmes_file = "Horizon 2020/cordisref-H2020programmes.xls"
h2020_programmes = pd.read_excel(
    h2020_programmes_file,
    sheet_name="Hoja1",
    header=0,
)

In [13]:
# Symbols and names of Horizon 2020 topics in several languages
h2020_topics_file = "Horizon 2020/cordisref-H2020topics.xlsx"
h2020_topics = pd.read_excel(
    h2020_topics_file,
    sheet_name="cordisref-H2020topics",
    header=0,
)

In [14]:
# Symbols and names of Horizon 2020 research topics
h2020_sic_file = "Horizon 2020/cordisref-sicCode.xls"
h2020_sic = pd.read_excel(
    h2020_sic_file,
    sheet_name="cordisref-sicCode",
    header=0,
)

### Horizon 2020 projects and organisations
The data for projects and participations in Horizon 2020 are available on the Open Data Portal of the European Union:

https://data.europa.eu/euodp/en/data/dataset/cordisH2020projects

The cut-off date is: 2017-10-12

Two datasets will be created:
* `h2020_proj`: with the descriptors of the Horizon 2020 funded projects
* `h2020_part`: with the descriptions of the participating organisations

These two datasets will be merged into one sigle `h2020` dataset where the project informations will be repeated to each participations.


In [15]:
%%time

h2020_proj_file = "Horizon 2020/cordis-h2020projects.csv"
h2020_part_file = "Horizon 2020/cordis-h2020organizations.xlsx"

h2020_proj = pd.read_csv(
    h2020_proj_file,
    sep=";",
    header=0,
)
h2020_part = pd.read_excel(
    h2020_part_file,
    sheet_name="organisation",
    header=0,
)

CPU times: user 6.6 s, sys: 24 ms, total: 6.62 s
Wall time: 6.62 s


In [16]:
# We  rename some columns which are in both datasets:
# In projects:
h2020_proj = h2020_proj.rename(
    index=str,
    columns={
        "id": "projectID",
        "rcn": "projectRCN",
        "acronym": "projectAcronym"
    }
)
# In participations:
h2020_part = h2020_part.rename(
    index=str,
    columns={
        "id": "organizationID",
        "projectRcn": "projectRCN",
        "name": "organizationName",
    }
)

Two projects with projectRCN have non-consistent project acronyms, so let's force them to the same writing:
* projectRCN = 208306
* projectRCN = 194607

In [17]:
h2020_part.loc[h2020_part.projectRCN.eq(208306)] =\
    h2020_proj.loc[h2020_proj.projectRCN.eq(208306)].projectAcronym[0]
h2020_part.loc[h2020_part.projectRCN.eq(194607)] =\
    h2020_proj.loc[h2020_proj.projectRCN.eq(194607)].projectAcronym[0]

So, let's merge these two datasets and create the `h2020` dataset.

In [18]:
h2020 = h2020_proj.merge(
    h2020_part,
    on=["projectRCN", "projectID", "projectAcronym"]
)

In [19]:
h2020_proj.shape, h2020_part.shape, h2020.shape

((14837, 21), (71312, 23), (62495, 41))

We can see from the fusion that not every project record number has found its corresponding record number in the other dataset. We find indeed 8806 project record numbers (projectRCN) in the `h2020_part` dataset that are _not_ matched in the `h2020_proj` dataset, which is exactly the difference between the size of `h2020_part` and `h2020`.

In [20]:
rcn_in_proj = h2020_proj["projectRCN"].unique()
missing_rcn = h2020_part.loc[~h2020_part["projectRCN"].isin(rcn_in_proj),"projectRCN"]

In [21]:
len(missing_rcn) + h2020.shape[0] == h2020_part.shape[0]

True

In [22]:
# We won't use these dataframes anymore
h2020_part = 0
h2020_proj = 0

### Merge all project and participations information
We list below all the column headers of the datasets.

In [23]:
h2020.columns == fp7.columns

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True])

We see that both datasets, `fp7` and `h2020`have the same set of columns. So let's merge these two datasets, `h2020` following `fp7`:

In [24]:
p = fp7.append(h2020)

# We won't use these dataframes anymore
fp7 = 0
h2020 = 0

In [25]:
p = p.set_index(["frameworkProgramme"])

In [26]:
print ("Column headers:\n- " + "\n- ".join(p.columns))

Column headers:
- projectRCN
- projectID
- projectAcronym
- status
- programme
- topics
- title
- startDate
- endDate
- projectUrl
- objective
- totalCost
- ecMaxContribution
- call
- fundingScheme
- coordinator
- coordinatorCountry
- participants
- participantCountries
- subjects
- role
- organizationID
- organizationName
- shortName
- activityType
- endOfParticipation
- ecContribution
- country
- street
- city
- postCode
- organizationUrl
- contactType
- contactTitle
- contactFirstNames
- contactLastNames
- contactFunction
- contactTelephoneNumber
- contactFaxNumber
- contactEmail


## International organisations in Switzerland

Extract the data for Switzerland, but without the following international organisations that are counted for Switzerland by the European Commission. We consider in this statistics:

- Organisation météorologique mondiale: WMO
- United Nations International Strategy for Disaster Reduction: UNISDR
- International Organization for Migration: IOM
- European Organization for Nuclear Research: CERN
- World Health Organization: FT
- Union International pour la Conservation de la Nature et de ses Ressources: IUCN
- International Centre for Trade and Sustainable Development: ICTSD
- European Society of Intensive Care Medecine: ESICM
- United Nations Institute for Training and Research: UNITAR
- Worl Heart Federation: WHF
- Quaker United Nations Office, Geneva: 
- Council on Health Research for Development Association: COHRED
- Union Européenne de Radio Télévision: EBU
- European Molecular Biology Organization: EMBO, which is sometimes located by the European Commission in Switzerland (Geneva)
- European Society for Medical Oncology: ESMO
- International Telecommunication Union: ITU


In [27]:
fp7 = p.loc["FP7"]
h2020 = p.loc["H2020"]

In [28]:
# List of international organisations located in Switzerland (short names)
international_orgs_in_CH = [
    "WMO","UNISDR","IOM", "ITU", # in 'PUB'
    "CERN", "FT", "IUCN", "ICTSD", "ESICM", "UNITAR", # in 'REC'
    "WHF", "QUNO", "COHRED", "EBU", "EMBO", "ESMO", # in 'OTH'
]
international_orgs_in_CH.sort()

print ("Participations of international organisations in Switzerland:\n")

orgs_ch = pd.DataFrame (index = international_orgs_in_CH)

for org in orgs_ch.index:
    orgs_ch.loc[org,"Part in FP7"] = fp7.loc[fp7['shortName'] == org].shape[0]
    orgs_ch.loc[org,"FP7 Funding M€"] = fp7.loc[
        fp7['shortName'] == org,
        "ecContribution"
    ].sum()/1000000
    orgs_ch.loc[org,"Part in H2020"] = h2020.loc[
        h2020['shortName'] == org
    ].shape[0]
    orgs_ch.loc[org,"H2020 Funding M€"] = h2020.loc[
        h2020['shortName'] == org,
        "ecContribution"
    ].sum()/1000000
orgs_ch

Participations of international organisations in Switzerland:



,Part in FP7,FP7 Funding M€,Part in H2020,H2020 Funding M€
CERN,115.0,126.864954,52.0,37.340125
COHRED,4.0,1.170203,1.0,0.000000
EBU,10.0,2.552620,2.0,0.414555
EMBO,4.0,24.473964,0.0,0.000000
ESICM,1.0,0.232286,0.0,0.000000
ESMO,1.0,0.057780,0.0,0.000000
FT,44.0,11.902131,10.0,4.733969
ICTSD,1.0,0.141800,0.0,0.000000
IOM,60.0,15.432281,5.0,1.711823
ITU,59.0,12.472875,21.0,8.208944


In [29]:
all_ch = p.loc[(p['country'] == 'CH')]

# Correction by removing the international organisations in Switzerland
ch = p.loc[
    (p['country'] == 'CH') & (
        # We exclude here all international organisations located in
        # Switzerland, see list above.
        # "~" with ".isin" implements a de facto "is not in"
        ~p['shortName'].isin(international_orgs_in_CH)
    )
]

In [30]:
ch_part = pd.DataFrame(
    index = [
        "All",
        "International organisations in CH",
        "Without international organisations"
    ],
    columns = ["FP7","H2020"]
)
for prog in ["FP7","H2020"]:
    ch_part.loc["All",prog] = all_ch.loc[prog].shape[0]
    ch_part.loc["International organisations in CH",prog] = \
        all_ch["shortName"].isin(international_orgs_in_CH).sum()
    ch_part.loc["Without international organisations",prog] = ch.loc[prog].shape[0]    

print("Swiss participations in the framework programmes:\n")
ch_part

Swiss participations in the framework programmes:



,FP7,H2020
All,4921,1566
International organisations in CH,258,258
Without international organisations,4726,1503


## Analysis

This parts is the general analysis of the participations dataset extracted from the Cordis project files (`fp7` and `h2020` datasets).

### Swiss participations in FP7

#### Reference
We compare our analysis to the official figures provided by the Swiss Confederation (State Secretariat for Education, Research and Innovation):

Source: [Swiss Participation in European Research Framework Programmes](https://www.sbfi.admin.ch/dam/sbfi/en/dokumente/2016/01/beteiligung_der_schweizandeneuropaeischenforschungsrahmenprogram.pdf.download.pdf/swiss_participationineuropeanresearchframeworkprogrammes.pdf)

#### Swiss participations in FP7

We select all participations where the country of the organisation's address is set to Switzerland, but where the organisation in not in the list of the international organisations listed above.

In [31]:
ch_all = fp7.loc[(fp7['country'] == 'CH')]

**Comment on the code:**
`~fp7['shortName'].isin(international_orgs_in_CH` identifies all rows in fp7 whose org_shortName is not in the list of international organisations in Switzerland.

In [32]:
# Correction by removing the international organisations in Switzerland
ch = fp7.loc[
    (fp7['country'] == 'CH') & (
        # We exclude here all international organisations located in
        # Switzerland, see list above.
        # "~" with ".isin" implements a de facto "is not in"
        ~fp7['shortName'].isin(international_orgs_in_CH)
    )
]

# Coordinations
# org_role == "hostInstitution" is taken as coordinator for ERC
# beneficiaries
fp7_coord = (
    (fp7['role'] == "coordinator") | (fp7['role'] == "beneficiary")
).sum()
ch_coord  = (
    (ch['role'] == "coordinator") |
    (ch['role'] == "beneficiary") |
    (ch['role'] == "hostInstitution")
).sum()

In [33]:
# General participation
print ("General statistics:")
print ("Total participations in FP7: {:7d}".format(fp7.shape[0]))
print ("Total coordinations in FP7:  {:7d}".format(fp7_coord))
print (
    "Total financial contributions in FP7: M€{:10.3f}".format(
        fp7["ecContribution"].sum()/1000000
    )
)

# Participations
print ("\nSwiss participations in FP7:")
print ("Number of Swiss participations: {:5d}".format(ch.shape[0]))
print ("Proportion of Swiss participations: {:4.2f}%".format(
    ch.shape[0]/fp7.shape[0]*100)
)

# Coordinations
print ("\nSwiss coordinations:")
print ("Number of Swiss coordinations: {:5d}".format(ch_coord))
print ("Proportion of Swiss coordinations: {:4.2f}%".format(
    ch_coord/fp7_coord*100)
)

# Financial contributions
print ("\nFinancial contributions to Switzerland:")
print ("Amount received: M€{a:10.3f}".format(
    a=ch["ecContribution"].sum()/1000000)
)
print ("Proportion of funding to Switzerland: {:4.2f}%".format(
    ch["ecContribution"].sum()/fp7["ecContribution"].sum()*100)
)

General statistics:
Total participations in FP7:  146021
Total coordinations in FP7:    22816
Total financial contributions in FP7: M€ 52308.999

Swiss participations in FP7:
Number of Swiss participations:  4726
Proportion of Swiss participations: 3.24%

Swiss coordinations:
Number of Swiss coordinations:  1378
Proportion of Swiss coordinations: 6.04%

Financial contributions to Switzerland:
Amount received: M€  2436.137
Proportion of funding to Switzerland: 4.66%
